<a href="https://colab.research.google.com/github/Savitanna/sberbank_data_analysis_06_07_21/blob/master/hw_4_timeseries_Savitskaya_AV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# HW4 - выполнила Савицкая А.В.
import datetime # для работы со временем 

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
pd.options.display.max_rows = 4

### Упражнение 1

* Скачайте с портала Открытых Данных производственный календарь России: https://data.gov.ru/opendata/7708660670-proizvcalendar
* На его основе определите соответствующий календарь в Pandas: `RussianBusinessCalendar(AbstractHolidayCalendar)`
* С помощью полученного в пункте выше календаря и модуля `pd.offsets` создайте DataFrame one-hot календарных признаков:
    * День недели
    * Месяц
    * Выходной день
    * Праздничный день
    * Предпраздничный рабочий день
    * Последний день месяца
    * Последний рабочий день месяца
    * Предновогодний выходной день

In [5]:
my_path_dir="/content/drive/My Drive/Colab Notebooks/data/"

ts=pd.read_csv(filepath_or_buffer = my_path_dir+"data-20191112T1252-structure-20191112T1247.csv", engine='python', \
               delimiter=',', encoding='cp1251', index_col='Год/Месяц')
#,usecols=['Год/Месяц','Январь'])
ts

,Январь,Февраль,Март,Апрель,Май,Июнь,Июль,Август,Сентябрь,Октябрь,Ноябрь,Декабрь,Всего рабочих дней,Всего праздничных и выходных дней,Количество рабочих часов при 40-часовой рабочей неделе,Количество рабочих часов при 36-часовой рабочей неделе,Количество рабочих часов при 24-часовой рабочей неделе
Год/Месяц,,,,,,,,,,,,,,,,,
1999,"1,2,3,4,6*,7,9,10,16,17,23,24,30,31","6,7,13,14,20,21,27,28","6,7,8,13,14,20,21,27,28","3,4,10,11,17,18,24,25,30*","1,2,3,4,8,9,10,15,16,22,23,29,30","5,6,11*,12,13,14,19,20,26,27","3,4,10,11,17,18,24,25,31","1,7,8,14,15,21,22,28,29","4,5,11,12,18,19,25,26","2,3,9,10,16,17,23,24,30,31","6,7,8,13,14,20,21,27,28","4,5,11,12,13,18,19,25,26,31*",251,114,2004,1807.2,1204.8
2000,"1,2,3,4,6*,7,8,9,15,16,22,23,29,30","5,6,12,13,19,20,26,27","4,5,7*,8,11,12,18,19,25,26","1,2,8,9,15,16,22,23,29,30","1,2,6,7,8*,9,13,14,20,21,27,28","3,4,10,11,12,17,18,24,25","1,2,8,9,15,16,22,23,29,30","5,6,12,13,19,20,26,27","2,3,9,10,16,17,23,24,30","1,7,8,14,15,21,22,28,29","4,5,7,11,12,18,19,25,26","2,3,9,10,11*,12,16,17,23,24,30,31",250,116,1995,1800.0,1200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024,"1,2,3,4,5,6,7,8,13,14,20,21,27,28","3,4,10,11,17,18,22*,23,24,25","2,3,7*,8,9,10,16,17,23,24,30,31","6,7,13,14,20,21,27,28,30*","1,4,5,8*,9,11,12,18,19,25,26","1,2,8,9,11*,12,15,16,22,23,29,30","6,7,13,14,20,21,27,28","3,4,10,11,17,18,24,25,31","1,7,8,14,15,21,22,28,29","5,6,12,13,19,20,26,27","2,3,4,9,10,16,17,23,24,30","1,7,8,14,15,21,22,28,29,31*",250,116,1994,1794.0,1194.0
2025,"1,2,3,4,5,6,7,8,11,12,18,19,25,26","1,2,8,9,15,16,22,23,24+","1,2,7*,8,9,10+,15,16,22,23,29,30","5,6,12,13,19,20,26,27,30*","1,3,4,8*,9,10,11,17,18,24,25,31","1,7,8,11*,12,14,15,21,22,28,29","5,6,12,13,19,20,26,27","2,3,9,10,16,17,23,24,30,31","6,7,13,14,20,21,27,28","4,5,11,12,18,19,25,26","1,2,3*,4,8,9,15,16,22,23,29,30","6,7,13,14,20,21,27,28,31*",249,116,1986,1786.8,1189.2


In [6]:
size_m=ts.shape
name_month=ts.columns[0:12]
name_year=ts.index
#name_year=ts[ts.columns[0]]
new_size=(name_year.shape[0])*(name_month.shape[0])
ts1=pd.DataFrame(index=range(new_size), columns=['Год','Месяц','Нерабочие/Рабочие дни'])
ts1

,Год,Месяц,Нерабочие/Рабочие дни
0,NaN,NaN,NaN
1,NaN,NaN,NaN
...,...,...,...
322,NaN,NaN,NaN
323,NaN,NaN,NaN


In [7]:
k=0
for i in name_year:
  for j in name_month:
      ts1.loc[k,'Год']=i
      ts1.loc[k,'Месяц']=j
      # рабочие дни из строки переводим в массив
      ts1.loc[k,'Нерабочие/Рабочие дни']=list(ts.loc[i, j].split(','))
      #ts1.loc[k,'Нерабочие/Рабочие дни']=list(ts.loc[ts[ts['Год/Месяц'] == i].index[0], j].split(','))
      k=k+1
ts1

,Год,Месяц,Нерабочие/Рабочие дни
0,1999,Январь,"[1, 2, 3, 4, 6*, 7, 9, 10, 16, 17, 23, 24, 30,..."
1,1999,Февраль,"[6, 7, 13, 14, 20, 21, 27, 28]"
...,...,...,...
322,2025,Ноябрь,"[1, 2, 3*, 4, 8, 9, 15, 16, 22, 23, 29, 30]"
323,2025,Декабрь,"[6, 7, 13, 14, 20, 21, 27, 28, 31*]"


In [8]:
# создаем массив, куда будем прикреплять итоговые нерабочие дни через concat
ts2=pd.DataFrame(columns=['Дата', 'Год','Месяц', 'День','Нерабочий день', 'Рабочий предпраздничный день *', 'Рабочий день **','Рабочий день - перенос +'])
ts2

,Дата,Год,Месяц,День,Нерабочий день,Рабочий предпраздничный день *,Рабочий день **,Рабочий день - перенос +


In [9]:
for i in ts1.index:
  day_arr=ts1.loc[i,'Нерабочие/Рабочие дни']
  for j in day_arr:
    # временный массив, который для каждого дня будем присоединять к основному через concat
    ts_add=pd.DataFrame(data=np.zeros((1,8),dtype=int) ,columns=['Дата', 'Год','Месяц','День','Нерабочий день', 'Рабочий предпраздничный день *', 'Рабочий день **','Рабочий день - перенос +'])
    ts_add['Год']=ts1.loc[i,'Год']
    ts_add['Месяц']=ts1.loc[i,'Месяц']
    if j.isdigit():
      day_j=j
      ts_add['Нерабочий день']=1
    elif j.find('**')!=-1:
      day_j=j[:-2]
      ts_add['Рабочий день **']=1
    elif j.find('*')!=-1:
      day_j=j[:-1]
      ts_add['Рабочий предпраздничный день *']=1
    elif j.find('+')!=-1:
      day_j=j[:-1]
      ts_add['Рабочий день - перенос +']=1
    ts_add['День']=day_j
    name_month_j=name_month.get_loc(ts1.loc[i,'Месяц'])+1
    ts_add['Дата']=pd.to_datetime((str(ts1.loc[i,'Год'])+'.'+str(name_month_j)+'.'+str(day_j)))
    #, format='%d-%m-%Y')
    ts2= pd.concat([ts2, ts_add], axis=0,ignore_index=True)
#ts2.set_index(ts2['Дата'])
ts2
#ts2[ts2['Рабочий день - перенос +']==1]
#ts2[ts2['Рабочий предпраздничный день *']==1]
#ts2[ts2['Рабочий день **']==1]
#name_month[month_ts2-1])].index[0]


,Дата,Год,Месяц,День,Нерабочий день,Рабочий предпраздничный день *,Рабочий день **,Рабочий день - перенос +
0,1999-01-01,1999,Январь,1,1,0,0,0
1,1999-01-02,1999,Январь,2,1,0,0,0
...,...,...,...,...,...,...,...,...
3276,2025-12-28,2025,Декабрь,28,1,0,0,0
3277,2025-12-31,2025,Декабрь,31,0,1,0,0


In [10]:
ts3=ts2
ts3.index=ts3['Дата'].values
#ts3.drop('Дата', axis=1, inplace=True)
ts3

,Дата,Год,Месяц,День,Нерабочий день,Рабочий предпраздничный день *,Рабочий день **,Рабочий день - перенос +
1999-01-01,1999-01-01,1999,Январь,1,1,0,0,0
1999-01-02,1999-01-02,1999,Январь,2,1,0,0,0
...,...,...,...,...,...,...,...,...
2025-12-28,2025-12-28,2025,Декабрь,28,1,0,0,0
2025-12-31,2025-12-31,2025,Декабрь,31,0,1,0,0


In [11]:
ts4=ts3[ts3['Нерабочий день']==1].copy()
rus_holidays=ts4.index
rus_holidays

DatetimeIndex(['1999-01-01', '1999-01-02', '1999-01-03', '1999-01-04',
               '1999-01-07', '1999-01-09', '1999-01-10', '1999-01-16',
               '1999-01-17', '1999-01-23',
               ...
               '2025-11-29', '2025-11-30', '2025-12-06', '2025-12-07',
               '2025-12-13', '2025-12-14', '2025-12-20', '2025-12-21',
               '2025-12-27', '2025-12-28'],
              dtype='datetime64[ns]', length=3132, freq=None)

In [12]:
ts5=ts3[ts3['Рабочий предпраздничный день *']==1].copy()
#ts5
rus_work_before_holidays=ts5.index
rus_work_before_holidays

DatetimeIndex(['1999-01-06', '1999-04-30', '1999-06-11', '1999-12-31',
               '2000-01-06', '2000-03-07', '2000-05-08', '2000-12-11',
               '2001-03-07', '2001-04-30',
               ...
               '2024-04-30', '2024-05-08', '2024-06-11', '2024-12-31',
               '2025-03-07', '2025-04-30', '2025-05-08', '2025-06-11',
               '2025-11-03', '2025-12-31'],
              dtype='datetime64[ns]', length=133, freq=None)

In [13]:
from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday, nearest_workday, EasterMonday
from pandas.tseries.offsets import DateOffset, Day

In [14]:
class RussianBusinessCalendar(AbstractHolidayCalendar):
    start_date = datetime.datetime(1999, 1, 1)
    end_date = datetime.datetime(2025, 12, 31)
    rules = [
             Holiday('Нерабочие дни в РФ', month=data_h.month, day=data_h.day, year=data_h.year) for data_h in rus_holidays
             ]
    
russian_busday = pd.offsets.CustomBusinessDay(weekmask='Mon Tue Wed Thu Fri Sat Sun', 
                                              calendar=RussianBusinessCalendar())
#russian_busday
calendar_RF = RussianBusinessCalendar()
calendar_RF.holidays()

DatetimeIndex(['1999-01-01', '1999-01-02', '1999-01-03', '1999-01-04',
               '1999-01-07', '1999-01-09', '1999-01-10', '1999-01-16',
               '1999-01-17', '1999-01-23',
               ...
               '2025-11-29', '2025-11-30', '2025-12-06', '2025-12-07',
               '2025-12-13', '2025-12-14', '2025-12-20', '2025-12-21',
               '2025-12-27', '2025-12-28'],
              dtype='datetime64[ns]', length=3132, freq=None)

In [15]:
class RussianBusinessCalendar_fiestas(AbstractHolidayCalendar):
    start_date = datetime.datetime(1999, 1, 1)
    end_date = datetime.datetime(2025, 12, 31)
    rules = [
             Holiday('Январские праздники', month=1, day=1),
             Holiday('Рождество Христово', month=1, day=7),
             Holiday('День защитника Отечества', month=2, day=23),
             Holiday('Международный женский день', month=3, day=8),
             Holiday('Праздник Весны и Труда', month=5, day=1),
             Holiday('День Победы', month=5, day=9),
             Holiday('День России', month=6, day=12),
             Holiday('День народного единства', month=11, day=4)
             ]
    
russian_busday_fiestas = pd.offsets.CustomBusinessDay(weekmask='Mon Tue Wed Thu Fri Sat Sun', calendar=RussianBusinessCalendar_fiestas())
russian_busday_fiestas


<CustomBusinessDay>

In [16]:
calendar_RF_fiestas = RussianBusinessCalendar_fiestas()
calendar_RF_fiestas.holidays()

DatetimeIndex(['1970-01-01', '1970-01-07', '1970-02-23', '1970-03-08',
               '1970-05-01', '1970-05-09', '1970-06-12', '1970-11-04',
               '1971-01-01', '1971-01-07',
               ...
               '2199-06-12', '2199-11-04', '2200-01-01', '2200-01-07',
               '2200-02-23', '2200-03-08', '2200-05-01', '2200-05-09',
               '2200-06-12', '2200-11-04'],
              dtype='datetime64[ns]', length=1848, freq=None)

In [17]:
class RussianBusinessCalendar_before_holidays(AbstractHolidayCalendar):
    start_date = datetime.datetime(1999, 1, 1)
    end_date = datetime.datetime(2025, 12, 31)
    rules = [
             Holiday('Предпраздничные дни в РФ', month=data_h_b.month, day=data_h_b.day, year=data_h_b.year) for data_h_b in rus_work_before_holidays
             ]
    
russian_busday_before_holidays = pd.offsets.CustomBusinessDay(weekmask='Mon Tue Wed Thu Fri Sat Sun', 
                                              calendar=RussianBusinessCalendar_before_holidays())
#russian_busday_before_holidays
calendar_RF_before_holidays = RussianBusinessCalendar_before_holidays()
calendar_RF_before_holidays.holidays()

DatetimeIndex(['1999-01-06', '1999-04-30', '1999-06-11', '1999-12-31',
               '2000-01-06', '2000-03-07', '2000-05-08', '2000-12-11',
               '2001-03-07', '2001-04-30',
               ...
               '2024-04-30', '2024-05-08', '2024-06-11', '2024-12-31',
               '2025-03-07', '2025-04-30', '2025-05-08', '2025-06-11',
               '2025-11-03', '2025-12-31'],
              dtype='datetime64[ns]', length=133, freq=None)

In [18]:
date_begin='2013-3-25'
date_end='2020-1-10'
new_size_2=pd.to_datetime(date_end)-pd.to_datetime(date_begin)
ts6 = pd.DataFrame(np.zeros((pd.date_range(start=date_begin, end=date_end, freq='D').shape[0],25)), \
                   columns=['выходной',	'праздник',	'предпраздничный рабочий день',	'последний день месяца',	'последний рабочий день месяца',	'пн',	'вт',	'ср',	'чт',	'пт',	'сб',	'вс',	'январь',	'февраль',	'март', 'апрель',	'май',	'июнь',	'июль',	'август',	'сентябрь',	'октябрь',	'ноябрь',	'декабрь',	'Предновогодний выходной день'], \
                   index=pd.date_range(start=date_begin, end=date_end, freq='D')) 
ts6

,выходной,праздник,предпраздничный рабочий день,последний день месяца,последний рабочий день месяца,пн,вт,ср,чт,пт,сб,вс,январь,февраль,март,апрель,май,июнь,июль,август,сентябрь,октябрь,ноябрь,декабрь,Предновогодний выходной день
2013-03-25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-03-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
name_day_arr=pd.Series(data=['пн',	'вт',	'ср',	'чт',	'пт',	'сб',	'вс'],
                          index=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
name_month_arr=pd.Series(data=['январь',	'февраль',	'март', 'апрель',	'май',	'июнь',	'июль',	'август',	'сентябрь',	'октябрь',	'ноябрь',	'декабрь'], 
                         index=['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'])
#name_day_arr['Monday']

In [21]:
# устанавливаем выходные дни
ts6.loc[ts6.index.isin(calendar_RF.holidays(date_begin, date_end)),'выходной']=1
# устанавливаем праздничные дни
ts6.loc[ts6.index.isin(calendar_RF_fiestas.holidays(date_begin, date_end)),'праздник']=1
# устанавливаем предпраздничные рабочие дни
ts6.loc[ts6.index.isin(calendar_RF_before_holidays.holidays(date_begin, date_end)),'предпраздничный рабочий день']=1
# последний день месяца
ts6.loc[ts6.index.is_month_end==True,'последний день месяца']=1
# последний рабочий день месяца
ts6.loc[(ts6['выходной']==0)&(ts6['последний день месяца']==1),'последний рабочий день месяца']=1 # где последний день месяца - рабочий
ts6.loc[ts6[(ts6['выходной']==1)&(ts6['последний день месяца']==1)].index-pd.offsets.BusinessDay(),'последний рабочий день месяца']=1 # где последний день месяца - НЕ рабочий
# дни недели
for i_day in name_day_arr.index:
  ts6.loc[ts6.index.day_name()==i_day, name_day_arr[i_day]]=1
# месяца
for i_month in name_month_arr.index:
  ts6.loc[ts6.index.month_name()==i_month, name_month_arr[i_month]]=1
# Предновогодний выходной день
ts6.loc[(ts6['выходной']==1)&(ts6.index.month==12)&(ts6.index.day==31),'Предновогодний выходной день']=1
# выводим итог
ts6

,выходной,праздник,предпраздничный рабочий день,последний день месяца,последний рабочий день месяца,пн,вт,ср,чт,пт,сб,вс,январь,февраль,март,апрель,май,июнь,июль,август,сентябрь,октябрь,ноябрь,декабрь,Предновогодний выходной день
2013-03-25,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-03-26,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# пример требуемого DataFrame 
events

,выходной,праздник,предпраздничный рабочий день,последний день месяца,последний рабочий день месяца,пн,вт,ср,чт,пт,сб,вс,январь,февраль,март,апрель,май,июнь,июль,август,сентябрь,октябрь,ноябрь,декабрь,Предновогодний выходной день
2014-01-01,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2014-01-02,1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2014-01-03,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2014-01-04,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2014-01-05,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0


### Упражнение 2

* Для временного ряда курса акций Сбербанка на основе производственного календаря России сформируйте DataFrame признаков:
    * Лаговые переменные рабочих дней с глубинами: 1, 3, 5
    * Скользящие статистики `['mean', 'median', 'max', 'min', 'std'] ` с окнами 1, 3, 5 сгрупированные по дням недели
    * Статистики `['mean', 'median', 'max', 'min', 'std'] ` в расширяющемся окне сгруппированные по рабочим, праздничным и предпраздничным дням

In [22]:
ts_sber=pd.read_csv(filepath_or_buffer = my_path_dir+"sber_stocks.csv", engine='python', \
                    usecols=['TRADEDATE', 'MARKETPRICE3'], )
ts_sber.index=ts_sber['TRADEDATE'].values
ts_sber.drop('TRADEDATE', axis=1, inplace=True)
ts_sber

,MARKETPRICE3
2013-03-25,99.99
2013-03-26,97.82
...,...
2020-01-09,259.23
2020-01-10,258.04


In [27]:
#ts_sber_d = ts_sber.join(ts6, how='outer')
ts_sber_d = ts6.join(ts_sber, how='left')
ts_sber_d

,выходной,праздник,предпраздничный рабочий день,последний день месяца,последний рабочий день месяца,пн,вт,ср,чт,пт,сб,вс,январь,февраль,март,апрель,май,июнь,июль,август,сентябрь,октябрь,ноябрь,декабрь,Предновогодний выходной день,MARKETPRICE3
2013-03-25,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99.99
2013-03-26,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,97.82
2013-03-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,96.45
2013-03-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,97.74
2013-03-29,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,98.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-06,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,252.64
2020-01-07,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2020-01-08,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,256.90
2020-01-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,259.23


In [24]:
pd.options.display.max_rows = 10

In [26]:
date_begin=ts_sber.index[0]
date_end=ts_sber.index[len(ts_sber.index)-1]
#date_begin
#date_end

In [28]:
ts_sber_d.loc[:, 'Price(-1)'] = ts_sber_d['MARKETPRICE3'].shift(1)
ts_sber_d.loc[:, 'Price(-3)'] = ts_sber_d['MARKETPRICE3'].shift(3)
ts_sber_d.loc[:, 'Price(-5)'] = ts_sber_d['MARKETPRICE3'].shift(5)
ts_sber_d

,выходной,праздник,предпраздничный рабочий день,последний день месяца,последний рабочий день месяца,пн,вт,ср,чт,пт,сб,вс,январь,февраль,март,апрель,май,июнь,июль,август,сентябрь,октябрь,ноябрь,декабрь,Предновогодний выходной день,MARKETPRICE3,Price(-1),Price(-3),Price(-5)
2013-03-25,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99.99,NaN,NaN,NaN
2013-03-26,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,97.82,99.99,NaN,NaN
2013-03-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,96.45,97.82,NaN,NaN
2013-03-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,97.74,96.45,99.99,NaN
2013-03-29,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,98.77,97.74,97.82,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-06,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,252.64,NaN,255.65,NaN
2020-01-07,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,252.64,NaN,NaN
2020-01-08,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,256.90,NaN,NaN,255.65
2020-01-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,259.23,256.90,252.64,NaN


In [40]:
# окно = 1
ts_sber_d.groupby(ts_sber_d.index.day_name()).rolling(window=1)['MARKETPRICE3', 'Price(-1)', 'Price(-3)', 'Price(-5)'].agg([np.mean, np.median, np.max, np.min, np.std])


MARKETPRICE3                  ... Price(-5)            
                             mean  median    amax  ...      amax    amin std
Friday    2013-03-29        98.77   98.77   98.77  ...       NaN     NaN NaN
          2013-04-05        99.29   99.29   99.29  ...       NaN     NaN NaN
          2013-04-12        99.14   99.14   99.14  ...       NaN     NaN NaN
          2013-04-19        96.34   96.34   96.34  ...       NaN     NaN NaN
          2013-04-26        96.04   96.04   96.04  ...       NaN     NaN NaN
...                           ...     ...     ...  ...       ...     ...  ..
Wednesday 2019-12-11       240.27  240.27  240.27  ...    233.65  233.65 NaN
          2019-12-18       243.90  243.90  243.90  ...    242.02  242.02 NaN
          2019-12-25       248.06  248.06  248.06  ...    244.54  244.54 NaN
          2020-01-01          NaN     NaN     NaN  ...    250.70  250.70 NaN
          2020-01-08       256.90  256.90  256.90  ...    255.65  255.65 NaN

[2483 rows x 20 columns]

In [41]:
# окно = 3
ts_sber_d.groupby(ts_sber_d.index.day_name()).rolling(window=3)['MARKETPRICE3', 'Price(-1)', 'Price(-3)', 'Price(-5)'].agg([np.mean, np.median, np.max, np.min, np.std])


MARKETPRICE3                  ... Price(-5)                  
                             mean  median    amax  ...      amax    amin       std
Friday    2013-03-29          NaN     NaN     NaN  ...       NaN     NaN       NaN
          2013-04-05          NaN     NaN     NaN  ...       NaN     NaN       NaN
          2013-04-12    99.066667   99.14   99.29  ...       NaN     NaN       NaN
          2013-04-19    98.256667   99.14   99.29  ...       NaN     NaN       NaN
          2013-04-26    97.173333   96.34   99.14  ...       NaN     NaN       NaN
...                           ...     ...     ...  ...       ...     ...       ...
Wednesday 2019-12-11   235.463333  234.94  240.27  ...    238.35  233.46  2.770024
          2019-12-18   238.450000  240.27  243.90  ...    242.02  233.46  4.888193
          2019-12-25   244.076667  243.90  248.06  ...    244.54  233.65  5.700868
          2020-01-01          NaN     NaN     NaN  ...    250.70  242.02  4.465393
          2020-01-08          NaN     NaN     NaN  ...    255.65  244.54  5.565971

[2483 rows x 20 columns]

In [42]:
# окно = 5
ts_sber_d.groupby(ts_sber_d.index.day_name()).rolling(window=5)['MARKETPRICE3', 'Price(-1)', 'Price(-3)', 'Price(-5)'].agg([np.mean, np.median, np.max, np.min, np.std])

  

MARKETPRICE3                  ... Price(-5)                  
                             mean  median    amax  ...      amax    amin       std
Friday    2013-03-29          NaN     NaN     NaN  ...       NaN     NaN       NaN
          2013-04-05          NaN     NaN     NaN  ...       NaN     NaN       NaN
          2013-04-12          NaN     NaN     NaN  ...       NaN     NaN       NaN
          2013-04-19          NaN     NaN     NaN  ...       NaN     NaN       NaN
          2013-04-26       97.916   98.77   99.29  ...       NaN     NaN       NaN
...                           ...     ...     ...  ...       ...     ...       ...
Wednesday 2019-12-11      236.452  237.64  240.27  ...    240.18  233.46  3.084790
          2019-12-18      237.704  238.23  243.90  ...    242.02  233.46  3.641726
          2019-12-25      239.670  240.27  248.06  ...    244.54  233.46  4.943999
          2020-01-01          NaN     NaN     NaN  ...    250.70  233.46  7.390141
          2020-01-08          NaN     NaN     NaN  ...    255.65  233.65  8.416357

[2483 rows x 20 columns]

In [43]:
# статистики в расширяющемся окне, сгруппированные по рабочим, праздничным и предпраздничным дням
# 
ts_sber_d.groupby(['выходной']).expanding(min_periods=1)['MARKETPRICE3', 'Price(-1)', 'Price(-3)', 'Price(-5)'].agg([np.mean, np.median, np.max, np.min, np.std])


MARKETPRICE3              ...  Price(-5)           
                            mean              ...        std           
                    MARKETPRICE3   Price(-1)  ...  Price(-3)  Price(-5)
выходной                                      ...                      
0.0      2013-03-25    99.990000         NaN  ...        NaN        NaN
         2013-03-26    98.905000   99.990000  ...        NaN        NaN
         2013-03-27    98.086667   98.905000  ...        NaN        NaN
         2013-03-28    98.000000   98.086667  ...        NaN        NaN
         2013-03-29    98.154000   98.000000  ...   1.534422        NaN
...                          ...         ...  ...        ...        ...
1.0      2020-01-04   154.915714  146.671335  ...  62.232634  61.916222
         2020-01-05   154.915714  146.671335  ...  62.232634  61.916222
         2020-01-06   157.630278  146.671335  ...  62.321501  61.916222
         2020-01-07   157.630278  146.948016  ...  62.321501  61.916222
         2020-01-08   160.313243  146.948016  ...  62.321501  62.008116

[2483 rows x 80 columns]

In [44]:
# 
ts_sber_d.groupby(['праздник']).expanding(min_periods=1)['MARKETPRICE3', 'Price(-1)', 'Price(-3)', 'Price(-5)'].agg([np.mean, np.median, np.max, np.min, np.std])


MARKETPRICE3              ...  Price(-5)           
                            mean              ...        std           
                    MARKETPRICE3   Price(-1)  ...  Price(-3)  Price(-5)
праздник                                      ...                      
0.0      2013-03-25    99.990000         NaN  ...        NaN        NaN
         2013-03-26    98.905000   99.990000  ...        NaN        NaN
         2013-03-27    98.086667   98.905000  ...        NaN        NaN
         2013-03-28    98.000000   98.086667  ...        NaN        NaN
         2013-03-29    98.154000   98.000000  ...   1.534422        NaN
...                          ...         ...  ...        ...        ...
1.0      2019-05-09          NaN  141.757419  ...  62.915720  53.164517
         2019-06-12          NaN  144.739375  ...  62.915720  55.554609
         2019-11-04          NaN  144.739375  ...  63.707879  57.098526
         2020-01-01          NaN  144.739375  ...  63.707879  58.890089
         2020-01-07          NaN  148.009091  ...  63.707879  58.890089

[2483 rows x 80 columns]

In [45]:
# 
ts_sber_d.groupby(['предпраздничный рабочий день']).expanding(min_periods=1)['MARKETPRICE3', 'Price(-1)', 'Price(-3)', 'Price(-5)'].agg([np.mean, np.median, np.max, np.min, np.std])


MARKETPRICE3  ...  Price(-5)
                                                mean  ...        std
                                        MARKETPRICE3  ...  Price(-5)
предпраздничный рабочий день                          ...           
0.0                          2013-03-25    99.990000  ...        NaN
                             2013-03-26    98.905000  ...        NaN
                             2013-03-27    98.086667  ...        NaN
                             2013-03-28    98.000000  ...        NaN
                             2013-03-29    98.154000  ...        NaN
...                                              ...  ...        ...
1.0                          2019-03-07   146.173750  ...  65.175145
                             2019-04-30   149.364400  ...  66.471544
                             2019-05-08   152.494615  ...  67.163326
                             2019-06-11   155.631111  ...  68.339302
                             2019-12-31   155.631111  ...  69.333375

[2483 rows x 80 columns]

In [46]:
# .rolling(window=5)
ts_sber_d.groupby(['выходной',	'праздник',	'предпраздничный рабочий день']).expanding(min_periods=1)['MARKETPRICE3', 'Price(-1)', 'Price(-3)', 'Price(-5)'].agg([np.mean, np.median, np.max, np.min, np.std])


MARKETPRICE3  ...  Price(-5)
                                                                  mean  ...        std
                                                          MARKETPRICE3  ...  Price(-5)
выходной праздник предпраздничный рабочий день                          ...           
0.0      0.0      0.0                          2013-03-25    99.990000  ...        NaN
                                               2013-03-26    98.905000  ...        NaN
                                               2013-03-27    98.086667  ...        NaN
                                               2013-03-28    98.000000  ...        NaN
                                               2013-03-29    98.154000  ...        NaN
...                                                                ...  ...        ...
1.0      1.0      0.0                          2019-05-09          NaN  ...  53.164517
                                               2019-06-12          NaN  ...  55.554609
                                               2019-11-04          NaN  ...  57.098526
                                               2020-01-01          NaN  ...  58.890089
                                               2020-01-07          NaN  ...  58.890089

[2483 rows x 80 columns]